In [1]:
import sys
sys.path.append('..')

In [2]:
from robustness_gym import *

In [30]:
import pandas as pd
import numpy as np
import yaml
import json
from fuzzywuzzy import fuzz
from transformers import *
from nlp import list_datasets

## Cached Operations

In [4]:
# Load a few examples from the boolq dataset
dataset = Dataset.load_dataset('boolq', split='train[:1%]')

Checking /Users/krandiash/.cache/huggingface/datasets/494f4f058948bc745482f513922716c8bff83799bb20c7026e85a0026b643401.8462e3fa824dc775b18d0c33eb06f61781c1157028a832923a3b9547ee06f9c2.py for additional imports.
Lock 140439929096896 acquired on /Users/krandiash/.cache/huggingface/datasets/494f4f058948bc745482f513922716c8bff83799bb20c7026e85a0026b643401.8462e3fa824dc775b18d0c33eb06f61781c1157028a832923a3b9547ee06f9c2.py.lock
Found main folder for dataset https://s3.amazonaws.com/datasets.huggingface.co/nlp/datasets/boolq/boolq.py at /usr/local/anaconda3/envs/mayanshell/lib/python3.8/site-packages/nlp/datasets/boolq
Found specific version folder for dataset https://s3.amazonaws.com/datasets.huggingface.co/nlp/datasets/boolq/boolq.py at /usr/local/anaconda3/envs/mayanshell/lib/python3.8/site-packages/nlp/datasets/boolq/f9eaaad11e850927735893d5fba78b4064e30569b9c0337facd7e35f6c0697a8
Found script file from https://s3.amazonaws.com/datasets.huggingface.co/nlp/datasets/boolq/boolq.py to /usr/

In [5]:
dataset

Dataset(schema: {'question': 'string', 'answer': 'bool', 'passage': 'string', 'index': 'string'}, num_rows: 94, num_slices: 0)

In [6]:
# Return a dataset which caches the outputs of the operations applied
dataset = dataset.stow(
    cached_ops={
        # Apply Spacy separately to only the "question" key
        Spacy(): [['question']],
        # Apply the StripText operation to the "question" key and the "passage" key separately
        StripText(): [['question'], ['passage']]
    },
    load_from_cache_file=False,
)

Caching processed dataset at cache-21673943725506890527705415222262047092225530433934578056354105899629.arrow
100%|██████████| 3/3 [00:00<00:00,  7.61it/s]
Done writing 94 examples in 158257 bytes cache-21673943725506890527705415222262047092225530433934578056354105899629.arrow.


In [9]:
print(dataset[0].keys())
print(dataset[0]['cache'].keys())
print(dataset[0]['cache']['Spacy'].keys())
print(dataset[0]['cache']['StripText'].keys())

dict_keys(['question', 'answer', 'passage', 'index', 'cache'])
dict_keys(['Spacy', 'StripText'])
dict_keys(['question'])
dict_keys(['passage', 'question'])


In [23]:
my_cached_op = CachedOperation(identifier='MyOp',  
                                apply_fn=lambda text_batch_a, text_batch_b: [fuzz.ratio(text_a, text_b) for text_a, text_b in zip(text_batch_a, text_batch_b)])

In [26]:
dataset = dataset.stow(cached_ops={
    my_cached_op: [['question', 'passage']],
})

Caching processed dataset at cache-25842721568737739515085703747208608204532832045481796505764655886292.arrow
100%|██████████| 3/3 [00:00<00:00, 50.15it/s]
Done writing 94 examples in 159009 bytes cache-25842721568737739515085703747208608204532832045481796505764655886292.arrow.


In [35]:
# The outputs are cached, as expected
print(dataset[0]['cache']['MyOp'])

{'["question", "passage"]': 13}


In [36]:
# The keys are JSON-dumped, so easy to load back in
for keys, value in dataset[0]['cache']['MyOp'].items():
    print(json.loads(keys), value)

['question', 'passage'] 13
